In [1]:
!pip install -q -U transformers datasets peft accelerate
# optional: keeps versions consistent
from google.colab import files


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 93.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.6/503.6 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 13.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.


In [2]:
!pip install -q -r requirements.txt


In [3]:
from data_loader import load_and_prepare, save_splits, SplitConfig

# fast subset for Colab (adjust to None for full dataset later)
cfg = SplitConfig(
    train_ratio=0.90, val_ratio=0.05, test_ratio=0.05,
    seed=42, max_train=5000, max_val=500, max_test=500
)

dsd = load_and_prepare(cfg)
paths = save_splits(dsd, "data/processed")
paths


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/944 [00:00<?, ?B/s]

data/train-00000-of-00001-297892d5d4e8a0(…):   0%|          | 0.00/36.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/52002 [00:00<?, ? examples/s]

Map:   0%|          | 0/52002 [00:00<?, ? examples/s]

Filter:   0%|          | 0/52002 [00:00<?, ? examples/s]

{'train': 'data/processed/train.jsonl',
 'validation': 'data/processed/validation.jsonl',
 'test': 'data/processed/test.jsonl'}

In [4]:
import json, itertools

def peek(path, n=3):
    with open(path, "r", encoding="utf-8") as f:
        for line in itertools.islice(f, n):
            print(json.loads(line)["prompt"][:300], "...")

print("TRAIN EXAMPLES:")
peek("data/processed/train.jsonl")
print("\nVAL EXAMPLES:")
peek("data/processed/validation.jsonl")


TRAIN EXAMPLES:
Instruction: Interpret the results of the MRI scan depicted in the question.
Input: An MRI scan demonstrates a hyperintense T2 lesion in the posterior aspect of the brain. Which of the following best describes this condition? A) Acoustic Neuroma B) Meningioma C) Glioblastoma D) Stroke

Response: ...
Instruction: Based on the provided details explain why pneumonia often leads to pleural effusion?
Input: <noinput>

Response: ...
Instruction: Choose the best answer that explains the following neurological condition.
Input: "A 55-year-old woman presents with progressively worsening tremors primarily in her right hand which becomes evident while holding objects. She has also noticed slowness in performing activities and mainta ...

VAL EXAMPLES:
Instruction: Translate the dissemination of medical information into layman's terms while keeping key points.
Input: "Osteoporosis is a systemic skeletal disease characterized by low bone mass and microarchitectural deterioration wit

In [5]:
!pip -q install -U transformers datasets peft accelerate
import torch, transformers, datasets, peft, accelerate
print("Torch:", torch.__version__)
print("Transformers:", transformers.__version__)


Torch: 2.8.0+cu126
Transformers: 4.57.0


In [6]:
from dataclasses import dataclass
from typing import Optional

@dataclass
class TrainConfig:
    model_name: str = "facebook/opt-1.3b"   # small, Colab-friendly
    train_path: str = "/content/data/processed/train.jsonl"
    val_path: str = "/content/data/processed/validation.jsonl"
    output_dir: str = "alpacare-lora"
    num_train_epochs: float = 1.0
    per_device_train_batch_size: int = 4
    per_device_eval_batch_size: int = 4
    gradient_accumulation_steps: int = 4
    learning_rate: float = 2e-4
    warmup_ratio: float = 0.03
    logging_steps: int = 50
    eval_steps: int = 200
    save_steps: int = 400
    fp16: bool = True   # for GPU memory savings

cfg = TrainConfig()

In [7]:
from datasets import load_dataset
import os

assert os.path.exists(cfg.train_path), "train.jsonl missing — run Day 1 first."
assert os.path.exists(cfg.val_path), "validation.jsonl missing."

train_ds = load_dataset("json", data_files=cfg.train_path, split="train")
val_ds = load_dataset("json", data_files=cfg.val_path, split="train")
train_ds = train_ds.shuffle(seed=42)
val_ds = val_ds.shuffle(seed=42)

print(train_ds[0])


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

{'prompt': 'Instruction: Summarize the main features and benefits of the given health insurance plan.\nInput: "The Gold Plan coverage includes hospital stays, ER visits, routine doctor\'s visits, outpatient surgery, lab tests, mental health services, home healthcare, physical therapy, orthopedics, allergy treatment, skin disorder treatment, vision care including glasses and contact lenses, dental exams cleanings and fillings, preventive care for children and adults at no additional cost. The plan has a $500 deductible and maximum out-of-pocket expense is $6,700.\n\nResponse:', 'response': "The main features and benefits of the Gold Plan health insurance coverage are:\n\n1. Hospital stays: This plan covers the expenses related to hospital stays, including inpatient care.\n\n2. ER visits: The plan provides coverage for emergency room visits, ensuring that the costs associated with urgent medical needs are taken care of.\n\n3. Routine doctor's visits: Regular visits to a primary care phys

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model

tokenizer = AutoTokenizer.from_pretrained(cfg.model_name, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(cfg.model_name)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj","v_proj"],
    lora_dropout=0.05,
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.63G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

trainable params: 3,145,728 || all params: 1,318,903,808 || trainable%: 0.2385


In [21]:
def format_example(example):
    text = example["prompt"] + "\n" + example["response"]
    return {"text": text}

train_fmt = train_ds.map(format_example, remove_columns=train_ds.column_names)
val_fmt = val_ds.map(format_example, remove_columns=val_ds.column_names)

def tokenize(examples):
    # Ensure padding is handled during tokenization
    out = tokenizer(
        examples["text"],
        truncation=True,
        max_length=1024,
        padding="max_length" # Explicitly pad during tokenization
    )
    out["labels"] = out["input_ids"].copy()
    return out

train_tok = train_fmt.map(tokenize, batched=True, remove_columns=train_fmt.column_names)
val_tok = val_fmt.map(tokenize, batched=True, remove_columns=val_fmt.column_names)

len(train_tok), len(val_tok)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

(5000, 500)

In [22]:
from transformers import DataCollatorForLanguageModeling, Trainer, TrainingArguments
import torch

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Move model to GPU if available and ensure all parameters are on the correct device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
for param in model.parameters():
    param.data = param.data.to(device)
if hasattr(model, 'buffers'):
    for buffer in model.buffers():
        buffer.data = buffer.data.to(device)


args = TrainingArguments(
    output_dir=cfg.output_dir,
    learning_rate=cfg.learning_rate,
    per_device_train_batch_size=cfg.per_device_train_batch_size,
    per_device_eval_batch_size=cfg.per_device_eval_batch_size,
    gradient_accumulation_steps=cfg.gradient_accumulation_steps,
    num_train_epochs=cfg.num_train_epochs,
    warmup_ratio=cfg.warmup_ratio,
    logging_steps=cfg.logging_steps,
    eval_steps=cfg.eval_steps, # eval_steps triggers evaluation when eval_dataset is provided
    save_steps=cfg.save_steps,
    fp16=cfg.fp16,
    report_to="none",
    remove_unused_columns=False,
    # torch_dtype=torch.float16 if cfg.fp16 else None, # Removed torch_dtype
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_tok,
    eval_dataset=val_tok, # Include evaluation dataset
    data_collator=data_collator,
)

trainer.train()
trainer.save_model(cfg.output_dir)
tokenizer.save_pretrained(cfg.output_dir)
print("✅ LoRA adapter saved to", cfg.output_dir)

OutOfMemoryError: CUDA out of memory. Tried to allocate 16.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 4.12 MiB is free. Process 2396 has 14.73 GiB memory in use. Of the allocated memory 14.59 GiB is allocated by PyTorch, and 15.26 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)